In [ ]:
#Preparando o script para que o algoritmo seja processado na placa do video do PC
import tensorflow as tf
from keras.models import Sequential  

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        #Alocar memória de vídeo de forma dinâmica
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        print(e)

#Agora defina o modelo
with tf.device('/GPU:0'):
    model = Sequential()
   
############################################### PREVISÃO PARA VALORES DIARIOS ###############################################################################################

#Preparando o algoritmo MLP para previsões da Eto diaria
import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler

#Carregar o banco de dados
dados = pd.read_csv('caminho_do_arquivo.csv', sep=';')  
dados['datona'] = pd.to_datetime(dados['datona'], format='%d/%m/%Y') #Converter para formato de data

#"datona" é como esta nomeada a coluna contendo as informações de data, altere essa variavel de acordo com a nomeação do seu banco de dados

print(dados.head())
print(dados.columns)

dados = dados.sort_values('datona')  #Ordenar os dados

def create_sequences(data, window_size):
    sequences = []
    for i in range(len(data) - window_size):
        sequence = data.iloc[i:i + window_size]
        sequences.append(sequence.values)
    return sequences

window_size = 30  #Tamanho da janela de previsão
sequences = create_sequences(dados, window_size)

#Converter para numpy array
sequences = np.array(sequences)

#Separar features e target
X = sequences[:, :-1, 1:]  #Features (removendo a coluna 'datona' e mantendo as outras)
y = sequences[:, -1, -1]  #Target 

#Dividir em conjunto de treinamento e teste
train_size = int(0.85 * len(sequences)) #O tamanho do conjunto de treino pode ser ajustado
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

#Verificando a quantidade de dados nos conjuntos de treino e teste
print(f"Quantidade de dados no conjunto de treinamento: {len(X_train)} amostras")
print(f"Quantidade de dados no conjunto de teste: {len(X_test)} amostras")
print(f"Quantidade de rótulos no conjunto de treinamento: {len(y_train)} amostras")
print(f"Quantidade de rótulos no conjunto de teste: {len(y_test)} amostras")

#Conversão para float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

#Definindo a arquitetura da rede
from keras.models import Sequential
from keras.layers import Flatten, Dense, Dropout
from keras import regularizers

model = Sequential()
model.add(Flatten(input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(256, activation='sigmoid', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(128, activation='sigmoid', kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.001)))
model.add(Dropout(0.5))
model.add(Dense(64, activation='sigmoid', kernel_regularizer=regularizers.l1(0.01)))
model.add(Dense(32, activation='sigmoid', kernel_regularizer=regularizers.l2(0.001)))
model.add(Dense(1))

#Compilando o modelo
model.compile(optimizer='rmsprop', loss='mean_squared_error')

#Adicionando a função early stopping para evitar o overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)  #Monitora a perda de validação e para o treinamento após 50 épocas se não houver melhoria

history = model.fit(X_train, y_train, epochs=1000, batch_size=90, validation_data=(X_test, y_test), callbacks=[early_stopping])  #Treina o modelo com parada antecipada


import matplotlib.pyplot as plt

#Plota o histórico de treinamento e validação
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='validation')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

#Faz as previsões com o conjunto de teste
y_pred = model.predict(X_test)

#Calcula o RMSE (Root Mean Squared Error)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print('RMSE:', rmse)

#Calcula o MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
print('MSE:', mse)

#Calcula o MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print('MAE:', mae)

#Número de dias futuros para prever
num_days = 15  #Valor ajustavel

#Define a sequência inicial para fazer as previsões
sequence = X_test[-1].reshape(1, X_test.shape[1], X_test.shape[2])

#Lista para armazenar as previsões diárias
daily_predictions = []

#Número de horas por dia (considerando 24 horas por dia)
hours_per_day = 24

for day in range(num_days):
    next_prediction = model.predict(sequence)
    daily_predictions.append(next_prediction[0, 0])
    sequence = np.roll(sequence, -hours_per_day, axis=1) 
    sequence[0, -hours_per_day:] = next_prediction

#Exibe as previsões para os próximos 15 dias (diariamente)
print("Previsões para os próximos 15 dias:")
print(daily_predictions)

import pandas as pd

#Cria um DataFrame com as previsões diárias
df_daily_predictions = pd.DataFrame(daily_predictions, columns=['Predictions'])

#Adiciona uma coluna com a data para cada previsão diária
start_date = dados['datona'].iloc[-1] + pd.Timedelta(days=1)  
date_range = pd.date_range(start=start_date, periods=len(daily_predictions), freq='D')
df_daily_predictions['Date'] = date_range

#Exibe as previsões diárias no DataFrame
print(df_daily_predictions)

#Exporta para um arquivo Excel
df_daily_predictions.to_excel('MLP_diario.xlsx', index=False)


############################################### PREVISÃO PARA VALORES HORARIOS ###############################################################################################


import pandas as pd
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras import regularizers
from sklearn.preprocessing import MinMaxScaler

#Carrega o banco de dados
dados = pd.read_csv('caminho_do_arquivo.csv', sep=';', parse_dates=['datona'], dayfirst=True)

print(dados.head())
print(dados.columns)

dados = dados.sort_values('datona')  #Ordenar os dados 

def create_sequences(data, window_size):
    sequences = []
    for i in range(len(data) - window_size):
        sequence = data.iloc[i:i + window_size]
        sequences.append(sequence.values)
    return sequences

window_size = 90  #Tamanho da janela
sequences = create_sequences(dados, window_size)

#Converte para numpy array
sequences = np.array(sequences)

#Separa features e target
X = sequences[:, :-1, 1:]  #Features (remover a coluna 'datona' e manter as outras)
y = sequences[:, -1, -1]  #Target 

#Dividindi em conjunto de treinamento e teste
train_size = int(0.8 * len(sequences)) #Valor ajustavel
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

#Verifica a quantidade de dados nos conjuntos de treino e teste
print(f"Quantidade de dados no conjunto de treinamento: {len(X_train)} amostras")
print(f"Quantidade de dados no conjunto de teste: {len(X_test)} amostras")
print(f"Quantidade de rótulos no conjunto de treinamento: {len(y_train)} amostras")
print(f"Quantidade de rótulos no conjunto de teste: {len(y_test)} amostras")


#Conversão para float32
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
y_train = y_train.astype('float32')
y_test = y_test.astype('float32')

#Definindo o modelo MLP
model = Sequential()
model.add(Flatten(input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(128, activation='sigmoid'))
model.add(Dense(64, activation='sigmoid'))
model.add(Dropout(0.3))
model.add(Dense(32, activation='sigmoid'))
model.add(Dense(1))


#Compilando o modelo
model.compile(optimizer='rmsprop', loss='mean_squared_error')

#Adicionando a função early stopping para evitar o overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=50)  #Monitora a perda de validação e para o treinamento após 50 épocas se não houver melhoria

history = model.fit(X_train, y_train, epochs=3000, batch_size=90, validation_data=(X_test, y_test), callbacks=[early_stopping])  #Treina o modelo com parada antecipada



from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_squared_error, mean_absolute_error
from math import sqrt

#Faz as previsões com o conjunto de teste
y_pred = model.predict(X_test)

#Calcula o RMSE (Root Mean Squared Error)
rmse = sqrt(mean_squared_error(y_test, y_pred))
print('RMSE:', rmse)

#Calcula o MSE (Mean Squared Error)
mse = mean_squared_error(y_test, y_pred)
print('MSE:', mse)

#Calcula o MAE (Mean Absolute Error)
mae = mean_absolute_error(y_test, y_pred)
print('MAE:', mae)

#Define o número de horas por dia
hours_per_day = 24

#Define o número de dias futuros para prever
num_days = 15

#Define a sequência inicial para fazer as previsões
sequence = X_test[-1].reshape(1, X_test.shape[1], X_test.shape[2])

#Lista para armazenar as previsões
predictions = []

for day in range(num_days):
    for hour in range(hours_per_day):
        next_prediction = model.predict(sequence)
        predictions.append(next_prediction[0, 0])  
        sequence = np.roll(sequence, -1, axis=1)  
        sequence[0, -1] = next_prediction 

#Exibe as previsões para os próximos 15 dias (cada dia com 24 horas)
print("Previsões para os próximos 15 dias (24 horas por dia):")
print(predictions)

#Exporta para um arquivo Excel
df_predictions.to_excel('MLP_horario.xlsx', index=False)